In [4]:
import gensim.downloader as api
wv=api.load("word2vec-google-news-300")

In [8]:
wv.similarity(w1="great",w2="")

0.057994794

In [62]:
import pandas as pd
import numpy as np
df = pd.read_csv('Downloads/spam (1).csv', encoding='latin1')
df = df.dropna(how="any", axis=1)
df.columns = ['target', 'message']

df.head()

,target,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [40]:
df.target.value_counts()

target
ham     4825
spam     747
Name: count, dtype: int64

In [49]:
min_samples=747
df_ham=df[df.target	=="ham"].sample(min_samples,random_state=2022)
df_spam=df[df.target	=="spam"].sample(min_samples,random_state=2022)

In [51]:
df_balanced=pd.concat([df_ham,df_spam],axis=0)
df_balanced.target.value_counts()

target
ham     747
spam    747
Name: count, dtype: int64

In [52]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['target'])

df['target_encoded'] = le.transform(df['target'])
df.head()

,target,message,target_encoded,preprocessed_message
0,ham,"Go until jurong point, crazy.. Available only ...",0,jurong point crazy available bugis n great wor...
1,ham,Ok lar... Joking wif u oni...,0,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry 2 wkly comp win FA Cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,0,u dun early hor u c
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,nah think go usf live


In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_balanced.preprocessed_message, 
    df_balanced.target_encoded	, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df_balanced.target_encoded	
) 

In [69]:
#1. create a pipeline object
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
clfrf= Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),        #using the ngram_range parameter 
     ('Random Forest', RandomForestClassifier())         
])

#2. fit with X_train and y_train
clfrf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clfrf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95       149
           1       0.99      0.92      0.95       150

    accuracy                           0.95       299
   macro avg       0.96      0.95      0.95       299
weighted avg       0.96      0.95      0.95       299



In [63]:
from sklearn.naive_bayes import MultinomialNB
clf=Pipeline([
    ('vectorizer_tfidf',TfidfVectorizer()),
    ('Multi NB',MultinomialNB())])

clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95       149
           1       0.94      0.95      0.95       150

    accuracy                           0.95       299
   macro avg       0.95      0.95      0.95       299
weighted avg       0.95      0.95      0.95       299



In [67]:
from sklearn.svm import LinearSVC
clfsvc = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),        #using the ngram_range parameter 
     ("SVM", LinearSVC())         
])

#2. fit with X_train and y_train
clfsvc.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clfsvc.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       149
           1       0.96      0.95      0.95       150

    accuracy                           0.95       299
   macro avg       0.95      0.95      0.95       299
weighted avg       0.95      0.95      0.95       299



In [ ]:
import pickle

with open('Sms_spam_ham_detection.pkl', 'wb') as f:
    pickle.dump(rf, f)  